# Pipeline ETL - Exploration et Visualisation

**Objectif** : Explorer et fusionner les 3 types de sources de données

**Sources** :
1. API RTE (consommation électrique)
2. Fichier CSV (jours fériés)
3. Web Scrapping (prix spot électricité)

## 0. Imports et Configuration

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

# Configuration plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
%matplotlib inline

print("Configuration OK")

In [ ]:
# Connexion base de données
database_type = os.getenv('DATABASE_TYPE', 'sqlite').lower()

if database_type == 'postgresql':
    host = os.getenv('POSTGRES_HOST', 'localhost')
    port = os.getenv('POSTGRES_PORT', '5432')
    db = os.getenv('POSTGRES_DB', 'rte_consommation')
    user = os.getenv('POSTGRES_USER', 'rte_user')
    password = os.getenv('POSTGRES_PASSWORD', 'rte_secure_password')
    conn_string = f'postgresql://{user}:{password}@{host}:{port}/{db}'
else:
    db_path = os.path.abspath('../database/rte_consommation.db')
    conn_string = f'sqlite:///{db_path}'

engine = create_engine(conn_string)
print(f"Connexion: {database_type}")

## 1. EXTRACT - Extraction des 3 Sources

### Source 1 : API RTE (Consommation Électrique)

In [ ]:
# Extraction consommation
df_conso = pd.read_sql('SELECT * FROM consommation ORDER BY datetime', engine)
df_conso['datetime'] = pd.to_datetime(df_conso['datetime'])

print("="*70)
print("SOURCE 1: API RTE - Consommation Electrique")
print("="*70)
print(f"Enregistrements: {len(df_conso)}")
print(f"Periode: {df_conso['datetime'].min()} -> {df_conso['datetime'].max()}")
print(f"Colonnes: {list(df_conso.columns)}")
print(f"\nApercu:")
df_conso.head()

In [ ]:
# Statistiques consommation
print("Statistiques consommation electrique:")
print(df_conso['mw_conso'].describe())

# Graphique consommation
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Serie temporelle
axes[0].plot(df_conso['datetime'], df_conso['mw_conso'], linewidth=0.8)
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Consommation (MW)')
axes[0].set_title('Serie temporelle - Consommation RTE')
axes[0].grid(True, alpha=0.3)

# Distribution
axes[1].hist(df_conso['mw_conso'], bins=30, edgecolor='black', alpha=0.7)
axes[1].axvline(df_conso['mw_conso'].mean(), color='r', linestyle='--', label='Moyenne')
axes[1].axvline(df_conso['mw_conso'].median(), color='g', linestyle='--', label='Mediane')
axes[1].set_xlabel('Consommation (MW)')
axes[1].set_ylabel('Frequence')
axes[1].set_title('Distribution Consommation')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Source 2 : Fichier CSV (Jours Fériés)

In [ ]:
# Extraction calendrier
df_calendrier = pd.read_sql('SELECT * FROM calendrier_feries ORDER BY datetime', engine)
df_calendrier['datetime'] = pd.to_datetime(df_calendrier['datetime'])

print("="*70)
print("SOURCE 2: Fichier CSV - Jours Feries et Vacances")
print("="*70)
print(f"Enregistrements: {len(df_calendrier)}")
print(f"Periode: {df_calendrier['datetime'].min()} -> {df_calendrier['datetime'].max()}")
print(f"Colonnes: {list(df_calendrier.columns)}")

# Statistiques
nb_feries = df_calendrier['est_ferie'].sum()
nb_vacances = df_calendrier['est_vacances'].sum()
print(f"\nJours feries: {nb_feries} heures ({nb_feries/24:.0f} jours)")
print(f"Vacances scolaires: {nb_vacances} heures ({nb_vacances/24:.0f} jours)")

print(f"\nApercu:")
df_calendrier.head()

In [ ]:
# Liste des jours feries
jours_feries = df_calendrier[df_calendrier['est_ferie'] == True][['datetime', 'nom_ferie']].drop_duplicates()
jours_feries['date'] = jours_feries['datetime'].dt.date
print("Jours feries dans la periode:")
print(jours_feries[['date', 'nom_ferie']].to_string(index=False))

### Source 3 : Web Scrapping (Prix Spot Électricité)

In [ ]:
# Extraction prix spot
df_prix = pd.read_sql('SELECT * FROM prix_spot_electricite ORDER BY datetime', engine)
df_prix['datetime'] = pd.to_datetime(df_prix['datetime'])

print("="*70)
print("SOURCE 3: Web Scrapping - Prix Spot Electricite")
print("="*70)
print(f"Enregistrements: {len(df_prix)}")
print(f"Periode: {df_prix['datetime'].min()} -> {df_prix['datetime'].max()}")
print(f"Colonnes: {list(df_prix.columns)}")
print(f"\nApercu:")
df_prix.head()

In [ ]:
# Statistiques prix spot
print("Statistiques prix spot:")
print(df_prix['prix_spot_eur_mwh'].describe())

# Graphiques prix spot
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Serie temporelle
axes[0].plot(df_prix['datetime'], df_prix['prix_spot_eur_mwh'], linewidth=0.8, color='green')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Prix Spot (EUR/MWh)')
axes[0].set_title('Serie temporelle - Prix Spot Electricite')
axes[0].grid(True, alpha=0.3)

# Distribution
axes[1].hist(df_prix['prix_spot_eur_mwh'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1].axvline(df_prix['prix_spot_eur_mwh'].mean(), color='r', linestyle='--', label='Moyenne')
axes[1].set_xlabel('Prix Spot (EUR/MWh)')
axes[1].set_ylabel('Frequence')
axes[1].set_title('Distribution Prix Spot')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. TRANSFORM - Fusion et Enrichissement

### Fusion 1/2 : Consommation + Prix Spot

In [ ]:
# Fusion API + Scrapping
df_merged = pd.merge(df_conso, df_prix, on='datetime', how='inner', suffixes=('', '_prix'))

print("Fusion 1/2: Consommation (API) + Prix Spot (Scrapping)")
print(f"Enregistrements avant fusion: {len(df_conso)} (conso) + {len(df_prix)} (prix)")
print(f"Enregistrements apres fusion: {len(df_merged)}")
print(f"Taux de matching: {len(df_merged)/len(df_conso)*100:.1f}%")
print(f"\nColonnes: {list(df_merged.columns)}")

In [ ]:
# Visualisation correlation Consommation vs Prix
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot
axes[0].scatter(df_merged['mw_conso'], df_merged['prix_spot_eur_mwh'], alpha=0.5)
axes[0].set_xlabel('Consommation (MW)')
axes[0].set_ylabel('Prix Spot (EUR/MWh)')
axes[0].set_title(f"Consommation vs Prix Spot (corr = {df_merged['mw_conso'].corr(df_merged['prix_spot_eur_mwh']):.3f})")
axes[0].grid(True, alpha=0.3)

# Series temporelles superposees
ax1 = axes[1]
ax2 = ax1.twinx()

ax1.plot(df_merged['datetime'], df_merged['mw_conso'], 'b-', label='Consommation', linewidth=0.8)
ax2.plot(df_merged['datetime'], df_merged['prix_spot_eur_mwh'], 'g-', label='Prix Spot', linewidth=0.8)

ax1.set_xlabel('Date')
ax1.set_ylabel('Consommation (MW)', color='b')
ax2.set_ylabel('Prix Spot (EUR/MWh)', color='g')
ax1.tick_params(axis='y', labelcolor='b')
ax2.tick_params(axis='y', labelcolor='g')
axes[1].set_title('Evolution Consommation et Prix Spot')
ax1.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Fusion 2/2 : + Calendrier (Jours Fériés)

In [ ]:
# Fusion avec calendrier
calendrier_cols = ['datetime', 'est_ferie', 'est_vacances', 'nom_ferie']
df_calendrier_mini = df_calendrier[calendrier_cols]

df_merged = pd.merge(df_merged, df_calendrier_mini, on='datetime', how='left', suffixes=('', '_cal'))

print("Fusion 2/2: + Calendrier (Fichier CSV)")
print(f"Enregistrements apres fusion: {len(df_merged)}")
print(f"\nColonnes finales: {list(df_merged.columns)}")

In [ ]:
# Nettoyer colonnes redondantes
cols_to_drop = [col for col in df_merged.columns if col.endswith('_prix') or col.endswith('_cal') or col == 'created_at']
df_merged = df_merged.drop(columns=cols_to_drop, errors='ignore')

# Remplir valeurs manquantes calendrier
df_merged['est_ferie'] = df_merged['est_ferie'].fillna(False).astype(int)
df_merged['est_vacances'] = df_merged['est_vacances'].fillna(False).astype(int)
df_merged['nom_ferie'] = df_merged['nom_ferie'].fillna('')

print(f"Colonnes apres nettoyage: {list(df_merged.columns)}")

### Features Engineering

In [ ]:
# Creation features temporelles
df_merged['heure'] = df_merged['datetime'].dt.hour
df_merged['jour_semaine'] = df_merged['datetime'].dt.dayofweek
df_merged['mois'] = df_merged['datetime'].dt.month
df_merged['jour_mois'] = df_merged['datetime'].dt.day
df_merged['est_weekend'] = (df_merged['jour_semaine'] >= 5).astype(int)

print("Features temporelles creees:")
print("  - heure (0-23)")
print("  - jour_semaine (0-6)")
print("  - mois (1-12)")
print("  - jour_mois (1-31)")
print("  - est_weekend (0/1)")

print(f"\nColonnes finales ({len(df_merged.columns)}): {list(df_merged.columns)}")

In [ ]:
# Apercu donnees fusionnees
df_merged.head(10)

## 3. Analyse des Données Fusionnées

In [ ]:
# Statistiques globales
print("="*70)
print("STATISTIQUES DONNEES FUSIONNEES")
print("="*70)
print(f"Total enregistrements: {len(df_merged)}")
print(f"Periode: {df_merged['datetime'].min()} -> {df_merged['datetime'].max()}")
print(f"\nJours feries: {df_merged['est_ferie'].sum()} heures ({df_merged['est_ferie'].sum()/24:.0f} jours)")
print(f"Vacances scolaires: {df_merged['est_vacances'].sum()} heures ({df_merged['est_vacances'].sum()/24:.0f} jours)")
print(f"Weekends: {df_merged['est_weekend'].sum()} heures ({df_merged['est_weekend'].sum()/24:.0f} jours)")
print(f"\nConsommation moyenne: {df_merged['mw_conso'].mean():.2f} MW")
print(f"Prix spot moyen: {df_merged['prix_spot_eur_mwh'].mean():.2f} EUR/MWh")
print("="*70)

In [ ]:
# Matrice de correlation
corr_cols = ['mw_conso', 'prix_spot_eur_mwh', 'heure', 'jour_semaine', 'mois', 'est_ferie', 'est_vacances', 'est_weekend']
correlation = df_merged[corr_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0, fmt='.3f', 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matrice de Correlation - Donnees Multi-Sources', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCorrelations avec consommation (mw_conso):")
print(correlation['mw_conso'].sort_values(ascending=False))

### Analyse par Heure

In [ ]:
# Consommation moyenne par heure
conso_par_heure = df_merged.groupby('heure').agg({
    'mw_conso': ['mean', 'std', 'min', 'max'],
    'prix_spot_eur_mwh': 'mean'
}).round(2)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Consommation par heure
hours = range(24)
conso_mean = conso_par_heure['mw_conso']['mean']
conso_std = conso_par_heure['mw_conso']['std']

axes[0].plot(hours, conso_mean, 'b-', linewidth=2, label='Moyenne')
axes[0].fill_between(hours, conso_mean - conso_std, conso_mean + conso_std, alpha=0.3, label='±1 std')
axes[0].set_xlabel('Heure du jour')
axes[0].set_ylabel('Consommation (MW)')
axes[0].set_title('Consommation Moyenne par Heure')
axes[0].set_xticks(range(0, 24, 2))
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Prix spot par heure
prix_mean = conso_par_heure['prix_spot_eur_mwh']['mean']
axes[1].plot(hours, prix_mean, 'g-', linewidth=2)
axes[1].set_xlabel('Heure du jour')
axes[1].set_ylabel('Prix Spot (EUR/MWh)')
axes[1].set_title('Prix Spot Moyen par Heure')
axes[1].set_xticks(range(0, 24, 2))
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Consommation et Prix par heure:")
print(conso_par_heure)

### Analyse Jours Fériés vs Normaux

In [ ]:
# Comparaison jours feries vs normaux
if df_merged['est_ferie'].sum() > 0:
    jours_normaux = df_merged[df_merged['est_ferie'] == 0]['mw_conso']
    jours_feries = df_merged[df_merged['est_ferie'] == 1]['mw_conso']
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Boxplot
    data_box = [jours_normaux, jours_feries]
    axes[0].boxplot(data_box, labels=['Jours Normaux', 'Jours Feries'])
    axes[0].set_ylabel('Consommation (MW)')
    axes[0].set_title('Consommation: Jours Normaux vs Feries')
    axes[0].grid(True, alpha=0.3)
    
    # Histogrammes
    axes[1].hist(jours_normaux, bins=20, alpha=0.5, label='Jours Normaux', edgecolor='black')
    axes[1].hist(jours_feries, bins=20, alpha=0.5, label='Jours Feries', edgecolor='black')
    axes[1].set_xlabel('Consommation (MW)')
    axes[1].set_ylabel('Frequence')
    axes[1].set_title('Distribution Consommation')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Consommation moyenne jours normaux: {jours_normaux.mean():.2f} MW")
    print(f"Consommation moyenne jours feries: {jours_feries.mean():.2f} MW")
    print(f"Difference: {jours_normaux.mean() - jours_feries.mean():.2f} MW ({((jours_normaux.mean() - jours_feries.mean())/jours_normaux.mean()*100):.1f}%)")
else:
    print("Pas de jours feries dans la periode")

### Analyse Weekend vs Semaine

In [ ]:
# Comparaison weekend vs semaine
semaine = df_merged[df_merged['est_weekend'] == 0]['mw_conso']
weekend = df_merged[df_merged['est_weekend'] == 1]['mw_conso']

plt.figure(figsize=(10, 5))
plt.boxplot([semaine, weekend], labels=['Semaine', 'Weekend'])
plt.ylabel('Consommation (MW)')
plt.title('Consommation: Semaine vs Weekend')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Consommation moyenne semaine: {semaine.mean():.2f} MW")
print(f"Consommation moyenne weekend: {weekend.mean():.2f} MW")
print(f"Difference: {semaine.mean() - weekend.mean():.2f} MW")

## 4. LOAD - Sauvegarde Données Enrichies

In [ ]:
# Sauvegarder en base
table_name = 'conso_enrichi_3sources'
df_merged.to_sql(table_name, engine, if_exists='replace', index=False)

# Verifier
count = pd.read_sql(f'SELECT COUNT(*) as total FROM {table_name}', engine).iloc[0]['total']
print(f"Table '{table_name}' creee avec {count} enregistrements")

In [ ]:
# Export CSV
csv_path = '../data/conso_enrichi_3sources.csv'
df_merged.to_csv(csv_path, index=False)
print(f"Export CSV: {csv_path}")
print(f"Taille fichier: {os.path.getsize(csv_path) / 1024:.2f} KB")

## 5. Résumé Final

In [ ]:
print("="*70)
print("RESUME PIPELINE ETL")
print("="*70)
print("\n1. EXTRACT - 3 sources differentes:")
print(f"   - API RTE: {len(df_conso)} enregistrements consommation")
print(f"   - Fichier CSV: {len(df_calendrier)} enregistrements calendrier")
print(f"   - Web Scrapping: {len(df_prix)} enregistrements prix spot")
print("\n2. TRANSFORM - Fusion et enrichissement:")
print(f"   - Fusion 1: Consommation + Prix spot")
print(f"   - Fusion 2: + Calendrier jours feries")
print(f"   - Features engineering: 5 features temporelles")
print(f"   - Nettoyage: valeurs manquantes remplies")
print("\n3. LOAD - Sauvegarde:")
print(f"   - Base de donnees: table '{table_name}'")
print(f"   - Fichier CSV: {csv_path}")
print("\nResultat final:")
print(f"   - Enregistrements: {len(df_merged)}")
print(f"   - Colonnes: {len(df_merged.columns)}")
print(f"   - Sources integrees: 3 (API + Fichier + Scrapping)")
print("\nTop 3 correlations avec consommation:")
correlations_sorted = correlation['mw_conso'].sort_values(ascending=False)
for i, (var, corr) in enumerate(list(correlations_sorted.items())[1:4], 1):
    print(f"   {i}. {var}: {corr:.3f}")
print("="*70)